In [ ]:
import pandas as pd
import requests as r
import re
from datetime import datetime
from bs4 import BeautifulSoup
import time
#import numpy as np
import json
import unicodedata
import pymysql
from sqlalchemy import create_engine

## #1 Get the list of materials

I will only work on French market. 

Final structure of url is as follow: `https://www.doctolib.fr/{specialite}/{ville|departement|region|pays}?page={number}`

What I need:
* List of specialties
* List of department, region

In [ ]:
# Get the list of specialities
url='https://www.doctolib.fr/specialities'

headers="user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
headers=dict([i.split(': ') for i in headers.split('\n')])

r.get(url,headers=headers)

In [ ]:
html=r.get(url,headers=headers).content
soup=BeautifulSoup(html)

list_of_specialties_slug=[i["href"].strip('/') for i in soup.select('h1+div a')]
list_of_specialties=[i.text for i in soup.select('h1+div a')]
list_of_specialties_slug

In [ ]:
## Get the list of departments
departments=pd.read_csv('../data/departements-francais.csv',sep="\t") 
departments.head()

In [ ]:
index_to_drop=departments.iloc[-2:].index
departments.drop(index_to_drop,axis=0,inplace=True)

In [ ]:
departments.columns=departments.columns.map(str.capitalize)

In [ ]:
list_of_region=set([str(unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore')).strip("b'").strip('"').lower().replace("'","-").replace(" ","-") for x in departments.loc[:,'Region']])
list_of_region=list(list_of_region)
list_of_region[0]


In [ ]:
list_of_department=[str(unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore')).strip("b'").strip('"').lower().replace("'","-").replace(" ","-") for x in departments.loc[:,'Nom']]
list_of_department[6]


## #2 Get database

In [ ]:
# Get the content for one page
base_url='https://www.doctolib.fr'
speciality=list_of_specialties_slug[0]
location='aisne'

final_url=f"{base_url}/{speciality}/{location}?page=9"

headers="""accept: application/json
accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
cache-control: no-cache
content-type: application/json; charset=utf-8
cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22name%22%3A%22Paris%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTEwVDExOjMwOjQ5LjM3MloiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--34c8fb3207d661d5f08f5fd3388cb219e6a450ca; _doctolib_session=Hhv7pWEDJwdnyCQyqr7ThfPEQcS62D9lKg7M%2Fvz8R0AIHipjT1Hu8SN0rhu51Urw8FXsHuPmX0L2UfmzU62ZifdY0%2Ful6siBegn1tBS9xIOE68s0L8WvzGo2NpZ3WtP8lg7du3f9lA%2BDUY9qvMjbXWQbsnHeE2GdX%2F46rLZZ%2FDvPRwaKV4ZcEYrliAJoVx%2BTeyLg634T5KloWv2A4kStpH7OzTg2A%2FQs8TpEaffKhJckYWtV7uE0BQYcDW14sU3N18VFz1X3RU3uXoAhQxw7%2FcQgncDD4sdPdsO%2FUY7ou2T0IOMmi%2FkJL%2B2sVlpGRgfv21I5BaJFK0fu9l3YgjDL5BuvupSKtAxlaw%3D%3D--jN4XvOBX1kwEvjNq--YuCqF8thsE1OjQaZOd%2B%2Fug%3D%3D
referer: https://www.doctolib.fr/
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
x-csrf-token: +AD1NNffpu2TUGR+i552cv/SJOdpo7KzgJOwVBys6r3/EBA3AwlvUidQ0R+FVeD3cUpNqeH+LYB2447FTjv7Rg=="""
headers=dict([i.split(': ') for i in headers.split('\n')])

# Succeed to reteive a json format for the results of the search
result=r.get(final_url,headers=headers).json()

In [ ]:
len(result['data']['doctors'])+len(result['data']['directory_doctors'])

In [ ]:
data=pd.json_normalize(result['data'])
data[['has_more','is_hub','search_scope','total_search_results']]


In [ ]:
data2=pd.json_normalize(result['data']['doctors'])
data2

In [ ]:
# Get the content with all pages
base_url='https://www.doctolib.fr'
speciality='stomatologue'
location='france'

df_final=pd.DataFrame()
i=0
doctors=True

while doctors:
    
    i+=1
    
    final_url=f"{base_url}/{speciality}/{location}?page={i}"
    print("final_url: ",final_url)
    
    headers=f"""accept: application/json
    accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
    cache-control: no-cache
    content-type: application/json; charset=utf-8
    cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22name%22%3A%22Paris%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTEwVDExOjMwOjQ5LjM3MloiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--34c8fb3207d661d5f08f5fd3388cb219e6a450ca; _doctolib_session=Hhv7pWEDJwdnyCQyqr7ThfPEQcS62D9lKg7M%2Fvz8R0AIHipjT1Hu8SN0rhu51Urw8FXsHuPmX0L2UfmzU62ZifdY0%2Ful6siBegn1tBS9xIOE68s0L8WvzGo2NpZ3WtP8lg7du3f9lA%2BDUY9qvMjbXWQbsnHeE2GdX%2F46rLZZ%2FDvPRwaKV4ZcEYrliAJoVx%2BTeyLg634T5KloWv2A4kStpH7OzTg2A%2FQs8TpEaffKhJckYWtV7uE0BQYcDW14sU3N18VFz1X3RU3uXoAhQxw7%2FcQgncDD4sdPdsO%2FUY7ou2T0IOMmi%2FkJL%2B2sVlpGRgfv21I5BaJFK0fu9l3YgjDL5BuvupSKtAxlaw%3D%3D--jN4XvOBX1kwEvjNq--YuCqF8thsE1OjQaZOd%2B%2Fug%3D%3D
    referer: https://www.doctolib.fr/{speciality}/{location}?page={1 if i<2 else i-1}
    user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
    x-csrf-token: +AD1NNffpu2TUGR+i552cv/SJOdpo7KzgJOwVBys6r3/EBA3AwlvUidQ0R+FVeD3cUpNqeH+LYB2447FTjv7Rg=="""
    headers=dict([i.strip().split(': ') for i in headers.split('\n')])
    
    result=r.get(final_url,headers=headers).json()
    #variable doctors allows to set the while loop to False when there is no more results
    doctors=(len(result['data']['doctors'])!=0) or (len(result['data']['directory_doctors'])!=0)
    
    if len(result['data']['doctors'])>0:
        data=pd.json_normalize(result['data']['doctors'])
        data['doctolib_profile']=True
        df_final=df_final.append(data)
    
    if len(result['data']['directory_doctors'])>0:
        data=pd.json_normalize(result['data']['directory_doctors'])
        data=data.rename(columns={'zip_code':'zipcode'})
        data['doctolib_profile']=False
        data['profile_id']=-1
        df_final=df_final.append(data)
            
    time.sleep(1)
    

In [ ]:
df_final.iloc[:,:].head()

In [ ]:
df_final.columns
# drop columns: 'id', 'is_directory','cloudinary_public_id','exact_match', 'priority_speciality'

In [ ]:
list_of_specialties_slug

In [ ]:
# Final scrapping for all specialities
base_url='https://www.doctolib.fr'
location='france'

df_final=pd.DataFrame()

for i in range(len(list_of_specialties_slug)):
    
    speciality=list_of_specialties_slug[i]
    i=0
    doctors=True
    print("speciality: ",speciality)
    
    while doctors:
    
        i+=1

        final_url=f"{base_url}/{speciality}/{location}?page={i}"

        headers=f"""accept: application/json
        accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
        cache-control: no-cache
        content-type: application/json; charset=utf-8
        cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22name%22%3A%22Paris%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTEwVDExOjMwOjQ5LjM3MloiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--34c8fb3207d661d5f08f5fd3388cb219e6a450ca; _doctolib_session=Hhv7pWEDJwdnyCQyqr7ThfPEQcS62D9lKg7M%2Fvz8R0AIHipjT1Hu8SN0rhu51Urw8FXsHuPmX0L2UfmzU62ZifdY0%2Ful6siBegn1tBS9xIOE68s0L8WvzGo2NpZ3WtP8lg7du3f9lA%2BDUY9qvMjbXWQbsnHeE2GdX%2F46rLZZ%2FDvPRwaKV4ZcEYrliAJoVx%2BTeyLg634T5KloWv2A4kStpH7OzTg2A%2FQs8TpEaffKhJckYWtV7uE0BQYcDW14sU3N18VFz1X3RU3uXoAhQxw7%2FcQgncDD4sdPdsO%2FUY7ou2T0IOMmi%2FkJL%2B2sVlpGRgfv21I5BaJFK0fu9l3YgjDL5BuvupSKtAxlaw%3D%3D--jN4XvOBX1kwEvjNq--YuCqF8thsE1OjQaZOd%2B%2Fug%3D%3D
        referer: https://www.doctolib.fr/{speciality}/{location}?page={1 if i<2 else i-1}
        user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
        x-csrf-token: +AD1NNffpu2TUGR+i552cv/SJOdpo7KzgJOwVBys6r3/EBA3AwlvUidQ0R+FVeD3cUpNqeH+LYB2447FTjv7Rg=="""
        headers=dict([i.strip().split(': ') for i in headers.split('\n')])

        result=r.get(final_url,headers=headers).json()
        #variable doctors allows to set the while loop to False when there is no more results
        doctors=(len(result['data']['doctors'])!=0) or (len(result['data']['directory_doctors'])!=0)

        if len(result['data']['doctors'])>0:
            data=pd.json_normalize(result['data']['doctors'])
            data['doctolib_profile']=True
            df_final=df_final.append(data)

        if len(result['data']['directory_doctors'])>0:
            data=pd.json_normalize(result['data']['directory_doctors'])
            data=data.rename(columns={'zip_code':'zipcode'})
            data['doctolib_profile']=False
            data['profile_id']=-1
            df_final=df_final.append(data)

        time.sleep(1)

In [ ]:
df_final.shape

In [ ]:
# Save the data in a database
username='root'
host='localhost'
database='doctolib_db'
password=''

engine=create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

In [ ]:
# reset index to have a unique id for each row
df_final=df_final.reset_index()

In [ ]:
# drop useless columns before saving the data
col_to_drop=['id','index','is_directory','cloudinary_public_id','exact_match']
df_final=df_final.drop(col_to_drop,axis=1)

In [ ]:
new_df=df_final.loc[:3]
new_df

In [ ]:
df_final.sample(100)

In [ ]:
# send data into the database
new_df.to_sql('doctors',engine,if_exists='replace',index=False)

## #3 Get availability

Workflow:
* Ask to the user for a list of doctors
* Get the list from the database created from previous part
* Get the enriched information for each profile
* Access to the availability data thanks to enriched information

Structure of the availability url: `https://www.doctolib.fr/availabilities.json?start_date=2020-04-10&visit_motive_ids=1235300&agenda_ids=202675&limit=4`

In [ ]:
# Ask the user what kind of doctors they are looking for
specialty_search=input('What kind of doctor are you looking for?\n')

#Ask the user where they would like to consult
location_search=input('Where would you like to find this doctor? (please enter a zipcode)\n')

In [ ]:
specialties_match=tuple(i for i in list_of_specialties if re.search(specialty_search,i))
specialties_match

In [ ]:
#query='SELECT name_with_title, speciality, concat(address,", ",zipcode," ",city) as postal_address, case when doctolib_profile = 1 then True else False end as has_doctolib_profile, link FROM doctors WHERE zipcode = {} and speciality like "%{}%"'.format(location_search,specialty_search)
query='SELECT name_with_title, speciality, concat(address,", ",zipcode," ",city) as postal_address, case when doctolib_profile = 1 then "True" else "False" end as has_doctolib_profile, link FROM doctors WHERE zipcode = {} and speciality in {}'.format(location_search, specialties_match)
db=pd.read_sql_query(query,engine)


In [ ]:
db

In [ ]:
link=db.link[0]
doctor_slug=link[(re.search('(.*/){2}',link).end()):]
print(link)
doctor_slug

In [ ]:
# Get enriched information for each profile (agendas_id and visit_motive_id)
url=f'https://www.doctolib.fr/booking/{doctor_slug}.json'

headers=f"""accept: application/json
accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
content-type: application/json; charset=utf-8
cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJ29CbGL9S6EcRkCVhgT7xCgM%22%2C%22name%22%3A%22Aisne%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTExVDE0OjQ5OjE3LjQ0M1oiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--86af30e089255e9939a367ff59042ed80385d9df; _doctolib_session=OJbFmCwXUFFExx3D7ArhPZAcTmY%2FiuV45MFME%2BOWOqZHxHN2fm33buQ4ZIXhJYZC34rS6Dkbj9faL3khcXcZCLxEqkNxi8BJMzNzmdRcnLkl0s7NvJC%2FFrsRBzXugfb6GQ%2FsNcOB8feIqEd0ZzbVUahP0UN%2FDbh1UVdODs6E46xVDBNkLSs%2BkATECtQpeB%2BC0dp373gQBoHeYovxDVk2bixGyapLNiucG3k7d5a130qPFJBmnThKAI4zW7No%2FyqQEKzKPjLeuVtyVcxah18v2UeIw2pa%2Bols0KF6Grt2TOOKBxgB5yBqhWHYQbDUq7jx6wMYDnnGZDQwCfWWmB%2FqikKsm9A3lhMCA2%2Bkk6uWL6K6%2F43Wzj9WmuvfHUH8XmyaAUJKwav7vmAnH0Ze3qnuXFdTmiR4lEs93k2nIEfKMcOz6V6Z%2FgMo23w%3D--a4gEopZ%2BrN8qgCQC--SboWN%2F0tvvy8vPqtrWicEA%3D%3D
referer: https://www.doctolib.fr/{link}
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
x-csrf-token: HPo1YOpT3IX4imTZ1vYTpcdC3kXw36ciB1dwvSxUZH8b6tBjPoUVOkyK0bjYPYUgSdq3C3iCOBHxJ04sfsN1hA=="""
headers=dict([i.strip().split(': ') for i in headers.split('\n')])

result=r.get(url,headers=headers).json()


In [ ]:
# Get the first visit_motives_id
visit_motive=pd.json_normalize(result['data']['visit_motives'][0])
visit_motive['id']

In [ ]:
# Get the first agenda_id
agendas=pd.json_normalize(result['data']['agendas'][0])
agendas['id']

In [ ]:
data_request=[str(visit_motive['id'][0]),str(agendas['id'][0])]

In [ ]:
data_request[0]

In [ ]:
date=datetime.today().strftime('%Y-%m-%d')
date

In [ ]:
# Access to the availability data thanks to enriched information
url=f"https://www.doctolib.fr/availabilities.json?start_date={date}&visit_motive_ids={data_request[0]}&agenda_ids={data_request[1]}&limit=4"

headers=f"""accept: application/json
accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
content-type: application/json; charset=utf-8
cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJ29CbGL9S6EcRkCVhgT7xCgM%22%2C%22name%22%3A%22Aisne%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTExVDE0OjQ5OjE3LjQ0M1oiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--86af30e089255e9939a367ff59042ed80385d9df; _doctolib_session=OJbFmCwXUFFExx3D7ArhPZAcTmY%2FiuV45MFME%2BOWOqZHxHN2fm33buQ4ZIXhJYZC34rS6Dkbj9faL3khcXcZCLxEqkNxi8BJMzNzmdRcnLkl0s7NvJC%2FFrsRBzXugfb6GQ%2FsNcOB8feIqEd0ZzbVUahP0UN%2FDbh1UVdODs6E46xVDBNkLSs%2BkATECtQpeB%2BC0dp373gQBoHeYovxDVk2bixGyapLNiucG3k7d5a130qPFJBmnThKAI4zW7No%2FyqQEKzKPjLeuVtyVcxah18v2UeIw2pa%2Bols0KF6Grt2TOOKBxgB5yBqhWHYQbDUq7jx6wMYDnnGZDQwCfWWmB%2FqikKsm9A3lhMCA2%2Bkk6uWL6K6%2F43Wzj9WmuvfHUH8XmyaAUJKwav7vmAnH0Ze3qnuXFdTmiR4lEs93k2nIEfKMcOz6V6Z%2FgMo23w%3D--a4gEopZ%2BrN8qgCQC--SboWN%2F0tvvy8vPqtrWicEA%3D%3D
referer: https://www.doctolib.fr/{link}
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
x-csrf-token: HPo1YOpT3IX4imTZ1vYTpcdC3kXw36ciB1dwvSxUZH8b6tBjPoUVOkyK0bjYPYUgSdq3C3iCOBHxJ04sfsN1hA=="""
headers=dict([i.strip().split(': ') for i in headers.split('\n')])

result=r.get(url,headers=headers).json()

In [ ]:
availabilities=pd.json_normalize(result)
availabilities.next_slot[0]

In [ ]:
db['next_availability']=availabilities.next_slot[0]

In [ ]:
db